In [1]:
# Importing Dependencies
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

import datetime as dt
import numpy as np
import pandas as pd
from flask import Flask, jsonify

In [7]:
# Setting up Database
engine = create_engine("sqlite:///hawaii_6.sqlite")
Base = automap_base()
Base.prepare(engine, reflect=True)

measurements = Base.classes.measurements
stations = Base.classes.stations
session = Session(engine)

In [ ]:
# Setting up Flask
app = Flask(__name__)
@app.route("/")
def welcome():
    return ("Available Routes:<br/>"
            "/api/v1.0/<br/>"
            "/api/v1.0/precipitation<br/>"
            "/api/v1.0/stations<br/>"
            "/api/v1.0/tobs")

@app.route("/api/v1.0/precipitation")
def precipitation():
    meas_query = session.query(measurements).group_by(measurements.date).order_by(measurements.date.desc()).all()
    dates = []
    prcp = []
    for measurements in meas_query:
        dates.append(str(measurements.date))
        prcp.append(measurements.prcp)
    return jsonify(dict(zip(all_date, all_prcp)))

@app.route("/api/v1.0/stations")
def stations():
    stations_query = session.query(Station.station).all()
    return jsonify(list(np.ravel(stations_query)))

@app.route("/api/v1.0/tobs")
def tobs():
    tobs_query = session.query(measurements.tobs).group_by(measurements.date).order_by(measurements.date.desc()).all()
    return jsonify(list(np.ravel(tobs_query)))

@app.route("/api/v1.0/<start_date>/")
def start_date(start_date):
    weather = session.query(func.min(measurements.tobs),\
                          func.max(measurements.tobs),\
                          func.avg(measurements.tobs))\
    .filter(measurements.date >= start_date).first()
    return jsonify({"min": weather[0], "weather": temps[1], "weather": temps[2]})

@app.route("/api/v1.0/<start_date>/<end_date>/")
def start_end(start_date):
    weather = session.query(func.min(measurements.tobs),\
                          func.max(measurements.tobs),\
                          func.avg(measurements.tobs))\
    .filter(measurements.date >= start_date, measurements.date <= end_date).first()
    return jsonify({"min": weather[0], "weather": temps[1], "weather": temps[2]})

if __name__ == '__main__':
    app.run(debug=True)